<a href="https://colab.research.google.com/github/bipinKrishnan/torchkeras/blob/master/functional_api.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pkbar

In [2]:
import torch
from torch import nn
from torch import optim
from torchsummary import summary as summary_
import pkbar

import warnings
warnings.filterwarnings('ignore')

# TorchKeras

In [3]:
def Input(size):
  return size

In [4]:
class Dense(nn.Module):
  def __init__(self, outputs, activation):
    super().__init__()
    self.outputs = outputs
    self.activation = activation

  def __call__(self, inputs):
    if type(inputs) == int:
      self.inputs_size = inputs
      self.layers = nn.Sequential(
        nn.Linear(self.inputs_size, self.outputs),
        self.activation
    )

      return self.layers

    else:
      self.inputs = inputs
      self.layers = list(inputs)
      self.layers.extend([nn.Linear(self.layers[-2].out_features, self.outputs), self.activation])

      self.layers = nn.Sequential(*self.layers)

      return self.layers

In [86]:
class Model():
  def __init__(self, inputs, outputs, device):
    self.input_size = inputs
    self.device = device
    self.model = outputs.to(self.device)

  def compile(self, optimizer, loss, metrics):
    self.opt = optimizer
    self.criterion = loss
    self.metrics = metrics

  def summary(self):
    summary_(self.model, (1, self.input_size), device=self.device)
    print("Device Type:", self.device)

  def fit(self, data_x, data_y, epochs):
    self.model.train()

    for epoch in range(epochs):
      print("Epoch {}/{}".format(epoch+1, epochs))
      progress = pkbar.Kbar(target=len(data_x), width=25)
      
      for i, (data, target) in enumerate(zip(data_x, data_y)):
        self.opt.zero_grad()

        train_out = self.model(data.to(self.device))
        loss = self.criterion(train_out, target.to(self.device))
        loss.backward()

        self.opt.step()

        progress.update(i, values=[("loss: ", loss.item())])

      progress.add(1)

  def evaluate(self, test_x, test_y):
    self.model.eval()
    correct, loss = 0.0, 0.0

    progress = pkbar.Kbar(target=len(test_x), width=25)

    for i, (data, target) in enumerate(zip(test_x, test_y)):
      out = self.model(data.to(self.device))
      loss += self.criterion(out, target.to(self.device))

      correct += ((torch.max(out, 1)[1]) == target.to(self.device)).sum()

      progress.update(i, values=[("loss", loss.item()/len(test_x)), ("acc", (correct/len(test_x)).item())])
    progress.add(1)


  def fit_generator(self, generator, epochs):
    self.model.train()

    for epoch in range(epochs):
      print("Epoch {}/{}".format(epoch+1, epochs))
      progress = pkbar.Kbar(target=len(generator), width=25)

      for i, (data, target) in enumerate(generator):
        self.opt.zero_grad()

        train_out = self.model(data.to(self.device))
        loss = self.criterion(train_out.squeeze(), target.to(self.device))
        loss.backward()

        self.opt.step()

        progress.update(i, values=[("loss: ", loss.item())])

      progress.add(1)
      

  def evaluate_generator(self, generator):
    self.model.eval()
    correct, loss = 0.0, 0.0

    progress = pkbar.Kbar(target=len(generator), width=25)

    for i, (data, target) in enumerate(generator):
      out = self.model(data.to(self.device))
      loss += self.criterion(out.squeeze(), target.to(self.device))

      correct += (torch.max(out.squeeze(), 1)[1] == target.to(self.device)).sum()

      progress.update(i, values=[("test_acc", (correct/len(generator)).item()), ("test_loss", loss.item()/len(generator))])

    progress.add(1)
      

# Test data

In [87]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

input = Input(size=1024)
x = Dense(54, nn.ReLU())(input)
y = Dense(34, nn.ReLU())(x)
z = Dense(43, activation=nn.ReLU())(y)
a = Dense(10, activation=nn.ReLU())(z)
b = Dense(1, activation=nn.Sigmoid())(a)

In [88]:
model = Model(input, b, 'cuda')
model.compile(optim.Adam(z.parameters(), lr=0.001), nn.BCELoss(), 'accuracy')

In [89]:
model.summary()

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                [-1, 1, 54]          55,350
              ReLU-2                [-1, 1, 54]               0
            Linear-3                [-1, 1, 34]           1,870
              ReLU-4                [-1, 1, 34]               0
            Linear-5                [-1, 1, 43]           1,505
              ReLU-6                [-1, 1, 43]               0
            Linear-7                [-1, 1, 10]             440
              ReLU-8                [-1, 1, 10]               0
            Linear-9                 [-1, 1, 1]              11
          Sigmoid-10                 [-1, 1, 1]               0
Total params: 59,176
Trainable params: 59,176
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.23
Estimated Tot

In [90]:
torch.manual_seed(42)

x = torch.rand((10, 1, 1024), dtype=torch.float)
y = torch.tensor((torch.rand(10, 1) < 0.5), dtype=torch.float)
#y = torch.tensor([[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]], dtype=torch.int64).reshape(-1, 1)

In [91]:
model.fit(x, y, 4)

Epoch 1/4
10/10 [=========================] - 0s 2ms/step - loss: : 0.6943
Epoch 2/4
10/10 [=========================] - 0s 2ms/step - loss: : 0.6751
Epoch 3/4
10/10 [=========================] - 0s 2ms/step - loss: : 0.6546
Epoch 4/4
10/10 [=========================] - 0s 3ms/step - loss: : 0.6284


In [92]:
model.evaluate(x, y)

10/10 [=========================] - 0s 1ms/step - loss: 0.3598 - acc: 0.1111    


# CIFAR100

In [93]:
import torch
import torchvision
from torchvision import transforms
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.optim as optim

import numpy as np
import matplotlib.pyplot as plt

In [ ]:
bs = 128

transform = transforms.Compose([transforms.Grayscale(num_output_channels=1),
                                transforms.Resize((1, 28*28)),
                                transforms.ToTensor()
])

trainset = torchvision.datasets.CIFAR100(root='./data', train=True, download=True, transform=transform)
testset = torchvision.datasets.CIFAR100(root='./data', train=False, download=True, transform=transform)

In [95]:
#images for feeding fit_generator dense network 
#must be resized to (1, 1, height*width*channel)
trainset[0][0].shape

torch.Size([1, 1, 784])

In [96]:
trainloader = DataLoader(trainset, batch_size=bs, shuffle=True)
testloader = DataLoader(testset, batch_size=bs)

In [97]:
input = Input(size=784)
x = Dense(1000, nn.ReLU())(input)
y = Dense(500, nn.ReLU())(x)
z = Dense(700, activation=nn.ReLU())(y)
a = Dense(200, activation=nn.ReLU())(z)
b = Dense(100, activation=nn.ReLU())(a)

model = Model(input, b, device)
model.compile(optim.Adam(b.parameters(), lr=0.001), nn.CrossEntropyLoss(), 'accuracy')
model.summary()

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1              [-1, 1, 1000]         785,000
              ReLU-2              [-1, 1, 1000]               0
            Linear-3               [-1, 1, 500]         500,500
              ReLU-4               [-1, 1, 500]               0
            Linear-5               [-1, 1, 700]         350,700
              ReLU-6               [-1, 1, 700]               0
            Linear-7               [-1, 1, 200]         140,200
              ReLU-8               [-1, 1, 200]               0
            Linear-9               [-1, 1, 100]          20,100
             ReLU-10               [-1, 1, 100]               0
Total params: 1,796,500
Trainable params: 1,796,500
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.04
Params size (MB): 6.85
Estimat

In [98]:
model.fit_generator(trainloader, 3)

Epoch 1/3
391/391 [=========================] - 13s 33ms/step - loss: : 4.5967
Epoch 2/3
391/391 [=========================] - 13s 33ms/step - loss: : 4.5844
Epoch 3/3
391/391 [=========================] - 13s 33ms/step - loss: : 4.5749


In [99]:
model.evaluate_generator(testloader)

79/79 [=========================] - 2s 31ms/step - test_acc: 2.0894 - test_loss: 2.3429


# CIFAR10

In [ ]:
bs = 64

transform = transforms.Compose([transforms.Grayscale(num_output_channels=1),
                                transforms.Resize((1, 28*28)),
                                transforms.ToTensor()
])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

In [101]:
#images for feeding fit_generator dense network 
#must be of the form (1, 1, height*width*channel)
trainset[0][0].shape

torch.Size([1, 1, 784])

In [102]:
trainloader = DataLoader(trainset, batch_size=bs, shuffle=True)
testloader = DataLoader(testset, batch_size=bs)

In [103]:
input = Input(size=784)
x = Dense(1024, nn.ReLU())(input)
y = Dense(800, nn.ReLU())(x)
z = Dense(300, activation=nn.ReLU())(y)
a = Dense(200, activation=nn.ReLU())(z)
b = Dense(10, activation=nn.ReLU())(a)

model = Model(input, b, 'cuda')
model.compile(optim.Adam(b.parameters(), lr=0.001), nn.CrossEntropyLoss(), 'accuracy')
model.summary()

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1              [-1, 1, 1024]         803,840
              ReLU-2              [-1, 1, 1024]               0
            Linear-3               [-1, 1, 800]         820,000
              ReLU-4               [-1, 1, 800]               0
            Linear-5               [-1, 1, 300]         240,300
              ReLU-6               [-1, 1, 300]               0
            Linear-7               [-1, 1, 200]          60,200
              ReLU-8               [-1, 1, 200]               0
            Linear-9                [-1, 1, 10]           2,010
             ReLU-10                [-1, 1, 10]               0
Total params: 1,926,350
Trainable params: 1,926,350
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.04
Params size (MB): 7.35
Estimat

In [104]:
model.fit_generator(trainloader, 3)

Epoch 1/3
782/782 [=========================] - 14s 18ms/step - loss: : 2.2989
Epoch 2/3
782/782 [=========================] - 14s 18ms/step - loss: : 2.2935
Epoch 3/3
782/782 [=========================] - 14s 18ms/step - loss: : 2.2767


In [105]:
model.evaluate_generator(testloader)

157/157 [=========================] - 2s 16ms/step - test_acc: 5.2690 - test_loss: 1.1446


# MNIST

In [106]:
bs = 32

transform = transforms.Compose([
                                transforms.Resize((1, 28*28)),
                                transforms.ToTensor()
])

trainset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
testset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)

In [107]:
#images for feeding fit_generator dense network 
#must be of the form (1, 1, height*width*channel)
trainset[0][0].shape

torch.Size([1, 1, 784])

In [108]:
trainloader = DataLoader(trainset, batch_size=bs, shuffle=True)
testloader = DataLoader(testset, batch_size=bs)

In [109]:
input = Input(size=784)
x = Dense(700, nn.ReLU())(input)
y = Dense(200, nn.ReLU())(x)
z = Dense(300, activation=nn.ReLU())(y)
a = Dense(100, activation=nn.ReLU())(z)
b = Dense(10, activation=nn.ReLU())(a)

model = Model(input, b, 'cuda')
model.compile(optim.Adam(b.parameters(), lr=0.001), nn.CrossEntropyLoss(), 'accuracy')
model.summary()

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1               [-1, 1, 700]         549,500
              ReLU-2               [-1, 1, 700]               0
            Linear-3               [-1, 1, 200]         140,200
              ReLU-4               [-1, 1, 200]               0
            Linear-5               [-1, 1, 300]          60,300
              ReLU-6               [-1, 1, 300]               0
            Linear-7               [-1, 1, 100]          30,100
              ReLU-8               [-1, 1, 100]               0
            Linear-9                [-1, 1, 10]           1,010
             ReLU-10                [-1, 1, 10]               0
Total params: 781,110
Trainable params: 781,110
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.02
Params size (MB): 2.98
Estimated T

In [110]:
model.fit_generator(trainloader, 3)

Epoch 1/3
1875/1875 [=========================] - 17s 9ms/step - loss: : 1.6028
Epoch 2/3
1875/1875 [=========================] - 17s 9ms/step - loss: : 1.3103
Epoch 3/3
1875/1875 [=========================] - 17s 9ms/step - loss: : 1.2229


In [111]:
model.evaluate_generator(testloader)

313/313 [=========================] - 2s 7ms/step - test_acc: 9.8077 - test_loss: 0.5984
